In [3]:
from flair.data import Sentence
from flair.nn import Classifier

# make a sentence
sentence = Sentence('George Washington went to Washington.')

# load the NER tagger
tagger = Classifier.load('ner-large')

# run NER over sentence
tagger.predict(sentence)

# print the sentence with all annotations
print(sentence)

2024-01-25 09:12:35,114 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence[6]: "George Washington went to Washington." → ["George Washington"/PER, "Washington"/LOC]


In [32]:
from docx import Document
import re
doc = Document('C:\\Users\\hechu\\Downloads\\1st_Meeting_3rd_Session_Open-ended_Working_Group_on_Reducing_Space_Threats.docx')
speeches = {}

def clean(text):
    # Remove timestamps
    clean_text = re.sub(r'\d{2}:\d{2}:\d{2}\s*', '', text)
    return clean_text


paragraphs = [clean(para.text) for para in doc.paragraphs if para.text.strip() != '']


# Define the path for the output text file
output_file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\output.txt'

# Write paragraphs to the text file
with open(output_file_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write('\n'.join(paragraphs))

txt_file.close()

In [44]:
# Read the text file
file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\output.txt'  # Update with your file path

with open(file_path, 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()

# Dictionary to store speeches
speeches = {}
current_speaker = None
previous_speaker = None

# Process each line
for line in lines:
    if re.match(r'Speaker \d+', line.strip()):
        previous_speaker = current_speaker
        current_speaker = line.strip()
        if current_speaker not in speeches:
            speeches[current_speaker] = ''
    else:
        speeches[current_speaker] += line.strip() + ' '

txt_file.close()

def preprocess_text(text):
    processed_text = re.sub(r'\b(\w+)\s+(\d+(?:\/\d+)*)(?!\w)', r'\1_\2', text)
    return processed_text

processed_speeches = {}

for speaker, speech in speeches.items():

    processed_speaker = preprocess_text(speaker)
    processed_speech = preprocess_text(speech)
    
    processed_speeches[processed_speaker] = processed_speech

print(processed_speeches)

{'Speaker_1': "International political context. These factors cannot be overlooked, but we must not risk falling into in action and lose hope. Instead of this, we should all invite the situation, commit to shouldering our collective responsibility, working together with an awareness of the urgency required in addressing the threat posed by space systems, as I pointed out at the beginning of this process, I am convinced that we will not be able to make. Progress. If we see this process as a struggle between different groups and interests, it's not a matter of West versus ES versus South versus north. Rather, this is a global issue because in some way or another, it affects all countries. And all persons that is particularly important to under score, with this in mind, despite the pessimism shown amongst many, we have seen constructive and interactive discussions at past sessions with participation from delegations from all regions and development levels in the space sphere. I heard that

In [54]:
# Write the consolidated speeches back to a file
output_file_path = 'C:\\Users\\hechu\\Desktop\\AICP\\consolidated_output.txt'  # Update with your desired output path
with open(output_file_path, 'w', encoding='utf-8') as file:
    for speaker, speech in speeches.items():
        file.write(f"{speaker}\n{speech}\n\n")
file.close()

In [43]:
import re

def preprocess_text(text):
    # 应用通用的预处理规则
    processed_text = re.sub(r'\b(\w+)\s+(\d+)(?!\w)', r'\1_\2', text)

    # 特定情况的还原: 如果不希望某些词后面跟数字被加下划线，可以在这里添加规则进行还原
    # 例如，将 "its_78th" 还原为 "its 78th"

    return processed_text

# 测试文本
text = "This is its 78th session. Another example is the 3rd attempt. aaa 45/1321"

# 处理文本
processed_text = preprocess_text(text)

print("Processed Text:", processed_text)

Processed Text: This is its 78th session. Another example is the 3rd attempt. aaa_45/1321


In [55]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import spacy

with open('C:\\Users\\hechu\\Desktop\\AICP\\consolidated_output.txt', 'r', encoding='utf-8') as file:
    text = file.read()

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
sentences = [sent.text.strip() for sent in doc.sents]

print(sentences)


['Speaker 1\nInternational political context.', 'These factors cannot be overlooked, but we must not risk falling into in action and lose hope.', 'Instead of this, we should all invite the situation, commit to shouldering our collective responsibility, working together with an awareness of the urgency required in addressing the threat posed by space systems, as I pointed out at the beginning of this process, I am convinced that we will not be able to make.', 'Progress.', "If we see this process as a struggle between different groups and interests, it's not a matter of West versus ES versus South versus north.", 'Rather, this is a global issue because in some way or another, it affects all countries.', 'And all persons that is particularly important to under score, with this in mind, despite the pessimism shown amongst many, we have seen constructive and interactive discussions at past sessions with participation from delegations from all regions and development levels in the space sphe

In [56]:
key = "d6b2dcdfeca648e1be584e334f7be7b5"
endpoint = "https://spacethreat.cognitiveservices.azure.com/"

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example function for recognizing entities from text
def entity_recognition(client):
    try:
        Ner = ""
        max_batch_size = 5
        for i in range(0, len(sentences), max_batch_size):
            batch = sentences[i:i + max_batch_size]
            documents = [{"id": str(idx), "text": sent} for idx, sent in enumerate(batch, start=i)]
            
            
            result = client.recognize_entities(documents=documents)
            for doc in result:
                if not doc.is_error:
                    
                    for entity in doc.entities:
                        Ner += f"Text: {entity.text}, Category: {entity.category}, SubCategory: {entity.subcategory}"
                else:
                    print(f"Error in document {doc.id}: {doc.error}")
        return Ner
    except Exception as err:
        print("Encountered exception. {}".format(err))
with open('C:\\Users\\hechu\\Desktop\\AICP\\annotated_output.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(entity_recognition(client))
